In [2]:
%matplotlib inline


# Second-level fMRI model: true positive proportion in clusters

This script showcases the so-called Notip procedure, in
which the proportion of true discoveries in arbitrary clusters is estimated.
The clusters can be defined from the input image, i.e. in a circular way, as
the error control accounts for arbitrary cluster selection.

Blain, Alexandre, Bertrand Thirion, and Pierre Neuvial. "Notip: Non-parametric True Discovery Proportion control for brain imaging." NeuroImage 260 (2022): 119492. doi:10.1016/j.neuroimage.2022.119492


## Fetch dataset
We download a list of left vs right button press contrasts from a
localizer dataset. Note that we fetch individual t-maps that represent the
BOLD activity estimate divided by the uncertainty about this estimate.



In [3]:
from nilearn.datasets import fetch_localizer_contrasts

n_subjects = 16
data = fetch_localizer_contrasts(
    ["left vs right button press"],
    n_subjects,
    get_tmaps=True,
    legacy_format=False,
)

In [4]:
from nilearn.input_data import NiftiMasker
smoothing_fwhm = 8.0
nifti_masker = NiftiMasker(smoothing_fwhm=smoothing_fwhm)

In [5]:
fmri_input = nifti_masker.fit_transform(data["cmaps"])

## Estimate second level model
We define the input maps and the design matrix for the second level model
and fit it.



In [6]:
import pandas as pd

second_level_input = data["cmaps"]
design_matrix = pd.DataFrame(
    [1] * len(second_level_input), columns=["intercept"]
)

Model specification and fit



In [9]:
from nilearn.glm.second_level import SecondLevelModel

second_level_model = SecondLevelModel(smoothing_fwhm=4.0)
second_level_model = second_level_model.fit(
    second_level_input, design_matrix=design_matrix
)

To estimate the contrast is very simple. We can just provide the column
name of the design matrix.



In [10]:
z_map = second_level_model.compute_contrast(output_type="z_score")

In [11]:
from posthoc_fmri import get_clusters_table_Notip
get_clusters_table_Notip(z_map, fmri_input, stat_threshold=3.5)

,Cluster ID,X,Y,Z,Peak Stat,Cluster Size (mm3),Minimum TDP (Notip)
0,1,42.0,-24.0,54.0,5.26,10503,0.80
1,1a,27.0,-21.0,66.0,5.16,,
2,1b,51.0,-15.0,54.0,5.10,,
3,1c,21.0,-24.0,72.0,4.97,,
4,2,-18.0,-54.0,-21.0,4.59,1566,0.14
5,3,6.0,-21.0,51.0,4.15,189,0.00
6,4,9.0,-9.0,63.0,4.11,243,0.00
7,5,39.0,-18.0,18.0,4.07,891,0.00
8,5a,42.0,-27.0,18.0,3.78,,
9,6,-30.0,-81.0,-9.0,3.97,162,0.00
